<a href="https://colab.research.google.com/github/anmolag190153/BCS_summer_project_SER/blob/main/Speech_emotion_recognition_gender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SPEECH EMOTIONS RECOGNITION PROJECT
## BCS-IITK

# Gender Classification using MLP model

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa
import librosa.display
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# from keras.callbacks import CSVLogger

# from google.colab import drive
# drive.mount('/content/drive')

In [7]:
def extract_feature(file):
    result = np.array([])
    mfcc = np.mean(librosa.feature.mfcc(file, sr = sr, n_mfcc = 40).T, axis = 0)
    result = np.hstack((result, mfcc))
    chroma = np.mean(librosa.feature.chroma_stft(file, sr=sr).T, axis = 0)
    result = np.hstack((result, chroma))
    mel = np.mean(librosa.feature.melspectrogram(file, sr=sr).T, axis = 0)
    result = np.hstack((result, mel))
    spectral_rolloff = np.mean(librosa.feature.spectral_rolloff(file, sr = sr).T, axis = 0)
    result=np.hstack((result, spectral_rolloff))
    return result

In [8]:
def build_fc_model(input_shape = (180, 1), num_class = 2):
    

    model=tf.keras.models.Sequential([tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Dense(512,activation='relu', input_shape = input_shape, kernel_regularizer = tf.keras.regularizers.l2(1e-2)),
                                    tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Dense(512, activation = 'relu', kernel_regularizer = tf.keras.regularizers.l2(1e-3)),
                                    tf.keras.layers.Dropout(0.1),
                                    tf.keras.layers.Dense(128, activation = 'relu'),
                                    tf.keras.layers.Dense(64, activation = 'relu'),
                                    tf.keras.layers.Dense(num_class, activation = 'softmax')
                                    ])

    return model

# Loading Data
We are using RAVDESS Dataset

In [9]:
sr=22100
path = '../Audio Datasets/'
k = 0
x_data = np.ndarray(shape = (24 * 60, 120000))
y_data = np.ndarray(shape = (24 * 60, 1))
for i,actor in enumerate(os.listdir(path)):
    path1 = os.path.join(path,actor)
    for j, file in enumerate(os.listdir(path1)):
        if file=='test':
            continue
        b = file[18:20]
        y_data[k,0]= int(b) % 2
        temp_path = os.path.join(path1, file)
        # print(temp_path)
        var, _ = librosa.load(temp_path)
        x_data[k, :var.shape[0]] = var[:]
        x_data[k,var.shape[0]:] = 0
        k = k + 1

# Extracting features
We are using mfccs, chroma frequency, mel spectogram and spectral centroid 

In [10]:
x=[]
for i in range(24*60):
    feature = extract_feature(x_data[i,:])
    x.append(feature)

# Scalling the data and splitting data into Test and train sets

In [18]:
x_train, x_test, y_train, y_test = train_test_split(np.array(x), y_data, test_size = 0.2, random_state = 9)

mean_mlp = np.mean(x_train,axis=0)
std_mlp = np.std(x_train,axis=0)
np.savetxt('mean_mlp.csv',mean_mlp)
np.savetxt('std_mlp.csv',std_mlp)
# scaler = StandardScaler()
# x_train = scaler.fit_transform(x_train)
# x_test = scaler.transform(x_test)

# Training the Model

In [12]:
model = build_fc_model()
model.compile(loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
# csv_logger = CSVLogger('model_history_gender.log', separator = ',', append = False)
history = model.fit(x_train, y_train, epochs = 500, batch_size = 256, validation_data = (x_test, y_test))

2021-07-25 17:36:41.262580: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/sahil/myfiles/auv/training/assing_1/work/devel/lib:/opt/ros/noetic/lib
2021-07-25 17:36:41.262606: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-07-25 17:36:41.262620: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (sahil-G5): /proc/driver/nvidia/version does not exist
2021-07-25 17:36:41.262800: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-07-25 17:36:41

Epoch 1/500
5/5 [==============================] - 1s 47ms/step - loss: 3.5460 - accuracy: 0.7917 - val_loss: 3.1251 - val_accuracy: 0.9132
Epoch 2/500
5/5 [==============================] - 0s 14ms/step - loss: 2.9329 - accuracy: 0.9531 - val_loss: 2.7223 - val_accuracy: 0.9549
Epoch 3/500
5/5 [==============================] - 0s 16ms/step - loss: 2.5753 - accuracy: 0.9792 - val_loss: 2.4112 - val_accuracy: 0.9722
Epoch 4/500
5/5 [==============================] - 0s 14ms/step - loss: 2.2751 - accuracy: 0.9870 - val_loss: 2.1010 - val_accuracy: 0.9861
Epoch 5/500
5/5 [==============================] - 0s 16ms/step - loss: 2.0131 - accuracy: 0.9896 - val_loss: 1.8667 - val_accuracy: 0.9861
Epoch 6/500
5/5 [==============================] - 0s 14ms/step - loss: 1.7787 - accuracy: 0.9896 - val_loss: 1.6387 - val_accuracy: 0.9896
Epoch 7/500
5/5 [==============================] - 0s 13ms/step - loss: 1.5609 - accuracy: 0.9965 - val_loss: 1.4374 - val_accuracy: 1.0000
Epoch 8/500
5/5 [===

In [13]:
model.save('Gender_classification_mlp_model_Team-4')

2021-07-25 17:38:34.484318: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: Gender_classification_mlp_model_Team-4/assets


In [14]:
result = model.evaluate(x_test, y_test)
print('The accuracy obtained on validation is ', result[1] * 100, '%')

9/9 [==============================] - 0s 3ms/step - loss: 0.0202 - accuracy: 1.0000
The accuracy obtained on validation is  100.0 %
